# Introduction to Optimum-Graphcore: BERT Fine-tuning on IPUs

<p align="center">
    <img src="https://github.com/huggingface/optimum-graphcore/blob/main/readme_logo.png?raw=true" />
</p>

## Optimum-Graphcore

🤗 Optimum Graphcore is the interface between the 🤗 Transformers library and [Graphcore IPUs](https://www.graphcore.ai/products/ipu).
It provides a set of tools enabling model parallelization and loading on IPUs, training and fine-tuning on all the tasks already supported by Transformers while being compatible with the Hugging Face Hub and every model available on it out of the box.

🤗 Optimum Graphcore was designed with one goal in mind: make training and evaluation straightforward for any 🤗 Transformers user while leveraging the complete power of IPUs.


## What is an Intelligence Processing Unit (IPU)?
Quote from the Hugging Face [blog post](https://huggingface.co/blog/graphcore#what-is-an-intelligence-processing-unit):
>IPUs are the processors that power Graphcore’s IPU-POD datacenter compute systems. This new type of processor is designed to support the very specific computational requirements of AI and machine learning. Characteristics such as fine-grained parallelism, low precision arithmetic, and the ability to handle sparsity have been built into our silicon.

> Instead of adopting a SIMD/SIMT architecture like GPUs, Graphcore’s IPU uses a massively parallel, MIMD architecture, with ultra-high bandwidth memory placed adjacent to the processor cores, right on the silicon die.

> This design delivers high performance and new levels of efficiency, whether running today’s most popular models, such as BERT and EfficientNet, or exploring next-generation AI applications.






## Setup

This notebook will demonstrate how to fine-tune a pre-trained BERT model with PyTorch on the Graphcore IPU-POD16 system using Optimum Graphcore. We will use a BERT-Large model and fine-tune on the SQuADv1 Question/Answering task.

We will show how to take a BERT model written in PyTorch from the Transformers library from HuggingFace and run it on Graphcore IPUs using Optimum Graphcore.

Requirements:

- Paperspace account with access to the Hugging Face IPU runtime
- Optimum Graphcore, installed below

The Poplar SDK environment and required IPU hardware are already enabled by Paperspace.

In [3]:
%pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.2.1
    Uninstalling pip-22.2.1:
      Successfully uninstalled pip-22.2.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install optimum[graphcore]

Note: you may need to restart the kernel to use updated packages.


We set Hugging Face transformers to version 4.20.0.

In [5]:
%pip install transformers==4.20.0 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 41.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 50.8 MB/s eta 0:00:00
  Using cached numpy-1.23.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 10.4 MB/s eta 0:00:00
  Using cached tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 29.2 MB/s eta 0:00:00
  Using cached filelock-3.7.1-py3-none-any.whl (10 kB)
  Using cached regex-2022.7.25-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (768 kB)
  Using cached huggingface_hub-0.8.1-py3-none-any.whl (101 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 37.2 MB/s eta 0:00:00
  Using cached charset_normalizer-2.1.0-py3-none-any.whl (39 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import optimum.graphcore
print(optimum.graphcore.__version__)

0.3.1


## Background


### BERT

BERT fine-tuning is when you train a BERT model on a supervised learning task on a relatively small amount of data, by using starting weights obtained from pre-training on a large, generic text corpus. Pre-training of BERT requires a lot of unlabelled data (for instance all of Wikipedia + thousands of books) and a lot of compute. It is expensive and time-consuming, but after pre-training BERT will have learned an extremely good language model that can be fine-tuned on downstream tasks with small amount of labeled data, achieving great results.


![bert.png](images/bert.png)


In this notebook we will fine-tune BERT (pre-trained on IPU with the Wikipedia dataset) on a question answering task called SQuAD. Then we will perform inference on the accompanying validation dataset.
 

In [7]:
# Import standard packages
import transformers
import torch
import torch.nn as nn
import numpy as np
from tqdm.notebook import trange, tqdm
from datasets import load_dataset, load_metric
import time
from pathlib import Path

# To run on IPU we import popart and poptorch packages
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

In [8]:
import warnings
warnings.filterwarnings("ignore")

## 1. Get the data


**What is SQuAD?**

> Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

From https://rajpurkar.github.io/SQuAD-explorer/

Basically you train a model to take a question and read a passage of text and predict the start and end positions of where that answer lies in the passage. The image below shows an example from the dataset:



(Source: [Rajpurkar GitHub](https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/Normans.html))

For the case of SQuADv1 there are no unanswerable questions in the dataset.

We use the 🤗 `datasets` package to automatically download the SQuAD dataset

In [9]:
datasets = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

The SQuAD dataset consists of pre-defined training and validation splits.

In [10]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

Each row in the data consists of a passage of text - `context` - a question about the passage - `question` - and the answer(s) to the question - `answers`. The latter consists of the text in the passage and the start position in the text.

Here is an example row:

In [11]:
datasets["train"][10016]

{'id': '56de5ef44396321400ee2861',
 'title': 'Institute_of_technology',
 'context': 'Institutes of technology in Venezuela were developed in the 1950s as an option for post-secondary education in technical and scientific courses, after the polytechnic French concepts. At that time, technical education was considered essential for the development of a sound middle class economy.',
 'question': 'What type of economy was technical education in Venezuela intended to support?',
 'answers': {'text': ['middle class'], 'answer_start': [274]}}

**How do we preprocess this data to train it with a deep learning model?**

We need to `tokenize` the text to turn it from words into numbers. This is done using `transformers.BertTokenizer`. Let's use this to tokenize a shortened version of the example above:

In [12]:
from squad_preprocessing import tokenizer

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
example = {"context": "Institutes of technology in Venezuela were developed in the 1950s",
           "question": "When were Institutes of technology developed?"}
tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=32,
        stride=16,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

In [14]:
tokenized_example.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

Let's look at the `input_ids`:

In [15]:
tokenized_example.input_ids[0]

[101,
 2043,
 2020,
 12769,
 1997,
 2974,
 2764,
 1029,
 102,
 12769,
 1997,
 2974,
 1999,
 8326,
 2020,
 2764,
 1999,
 1996,
 4856,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [16]:
tokenizer.decode(tokenized_example.input_ids[0])

'[CLS] when were institutes of technology developed? [SEP] institutes of technology in venezuela were developed in the 1950s [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

As you can see in the decoded version, the question is placed at the start followed by a `[SEP]` token, then the context, followed by padding if required.

In [17]:
from squad_preprocessing import prepare_train_features, prepare_validation_features, tokenizer

In [18]:
train_dataset = datasets["train"].map(
    prepare_train_features,
    batched=True,
    num_proc=1,
    remove_columns=datasets["train"].column_names,
    load_from_cache_file=True,
)

# Create validation features from dataset
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    num_proc=1,
    remove_columns=datasets["validation"].column_names,
    load_from_cache_file=True,
)

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

## 2. Get the BERT model from `transformers`

Create the model on the host. We can use `from_pretrained` to load pretrained checkpoints from the HuggingFace Hub.

In [19]:
model = transformers.BertForQuestionAnswering.from_pretrained("Graphcore/bert-large-uncased")

Downloading:   0%|          | 0.00/703 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642M [00:00<?, ?B/s]

Some weights of the model checkpoint at Graphcore/bert-large-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from 

**Now we are ready to stick it in Optimum!**


> We can now set up our pipelined execution by specifying which layers to put on each IPU, and passing it to the `parallelize` method that we defined above.

> We also call the `.half()` method to cast all the model weights to half-precision (FP16). The `.train()` sets the PyTorch model to training mode.

> If you unfamiliar with training in half precision on IPU, then we have a tutorial on [Half and Mixed Precision in Poptorch](https://github.com/graphcore/tutorials/tree/master/tutorials/pytorch/tut3_mixed_precision).

## 3. How `optimum-graphcore` runs models on IPU


`optimum-graphcore` will run the model on IPU using both **pipelining** and **data parallelism** in order to maximise hardware use.

### Parallelism through pipelining

The model layers are split over 8 IPUs. We then use [*pipeline parallelism*](https://docs.graphcore.ai/projects/tf-model-parallelism/en/latest/pipelining.html) over the IPUs with gradient accumulation. We subdivide the compute batch into micro-batches that pass through the pipeline in the forward pass and then come back again in the backwards pass, accumulating gradients for the parameters as they go.

A complete pipeline step has a ramp-up phase the start and a ramp-down phase at the end. Increasing the gradient accumulation factor, increases the total batch size and also increases the pipeline efficiency, and therefore throughput, because the proportion of time in ramp-up/down phases will be reduced. 

![pipelining.png](images/pipelining.png)

### Partitioning the Model

BERT Large has 24 transformer layers, which we will split over our 4 IPUs. The position and word embeddings, and the first 3 encoder layers will sit on IPU0, the following 3 IPUs have 7 transformer layers each. This partition is specified in the `IPUConfig` by the `layers_per_ipu` parameter.

<img src="https://docs.graphcore.ai/projects/bert-training/en/latest/_images/bert-pipelining.png" width="500" />

### Data Parallelism

An IPU-POD16 contains 16 IPUs and our pipeline is 4 IPUs long. We can therefore replicate the pipeline, feeding four different micro-batches to the device, which multiplies the effective mini-batch size by four. We call this configuration a "4x4 pipeline".

### Recomputation Checkpoints

We can make more efficient use of the valuable In-Processor-Memory by saving only selected activation inputs and recomputing the rest. This lets us optimise on memory savings (by not storing all activations) vs FLOP expenditure (by not having to recompute all activations). 

<img src="images/recomputation.png" width="800" />

Source: [TensorFlow Model Parallelism: Recomputation](https://docs.graphcore.ai/projects/tf-model-parallelism/en/latest/pipelining.html#recomputation)

Checkpoints are automatically placed between each pipeline stage. In addition to these automatic checkpoints, we are adding one at the end of every transformer layer, which leads to better performance.

### Replicated Tensor Sharding of Optimizer State

As we are using multiple replicas (4 here), we can also distribute our optimizer state to reduce local memory usage, a method called [On-chip Replicated Tensor Sharding (RTS)](https://docs.graphcore.ai/projects/graphcore-glossary/en/latest/index.html#term-Replicated-tensor-sharding).

> To further improve memory availability we also have the option to store tensors in the POD-IPU16 Streaming Memory at the cost of increased communications.

![rts.png](images/rts.png)

## 3. Running with Optimum-Graphcore

To use `optimum-graphcore` are three main classes one needs to know:
- `IPUTrainer`: the trainer class that takes care of compiling the model to run on IPUs, and of performing training and evaluation.
- `IPUTrainingArguments`: the parameters for how the model will be trained by the trainer.
- `IPUConfig`: the class that specifies attributes and configuration parameters to compile and put the model on the device.

The `IPUTrainer` is very similar to the [🤗 Transformers Trainer](https://huggingface.co/docs/transformers/main_classes/trainer), and adapting a script using the Trainer to make it work with IPUs will mostly consists of simply swapping the `Trainer` class for the `IPUTrainer` one.

The `IPUTrainingArguments` is also very similar to the [🤗 Transformers TrainingArguments](https://huggingface.co/docs/transformers/v4.20.1/en/main_classes/trainer#transformers.TrainingArguments) with a few extra arguments for IPUs. 

In [20]:
ipu_config = IPUConfig.from_pretrained("Graphcore/bert-large-ipu", inference_device_iterations=4)

Downloading:   0%|          | 0.00/692 [00:00<?, ?B/s]

In [21]:
ipu_config

IPUConfig {
  "decompose_grad_sum": false,
  "device_iterations": 1,
  "embedding_serialization_factor": 2,
  "enable_half_first_order_momentum": true,
  "enable_half_partials": true,
  "executable_cache_dir": "./exe_cache",
  "execute_encoder_on_cpu_for_generation": false,
  "gradient_accumulation_steps": 16,
  "inference_device_iterations": 4,
  "inference_replication_factor": {
    "default": 1,
    "pod16": 4,
    "pod32": 8,
    "pod4": 1,
    "pod64": 16,
    "pod8": 2
  },
  "ipus_per_replica": 4,
  "layers_per_ipu": [
    3,
    7,
    7,
    7
  ],
  "matmul_proportion": [
    0.15,
    0.18,
    0.2,
    0.25
  ],
  "optimizer_state_offchip": true,
  "optimum_version": "1.3.0",
  "output_mode": "final",
  "profile_dir": "",
  "recompute_checkpoint_every_layer": true,
  "replicated_tensor_sharding": true,
  "replication_factor": {
    "default": 1,
    "pod16": 4,
    "pod32": 8,
    "pod4": 1,
    "pod64": 16,
    "pod8": 2
  },
  "seed": null,
  "sharded_execution_for_infere

`device_iterations` is the number of batches the device should run before returning to the user. Increasing `device_iterations` can more efficient because the loop runs on the IPU directly, reducing overhead costs. Please see the [documentation](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latest/batching.html?highlight=device%20iterations#poptorch-options-deviceiterations) for more information.

In [22]:
global_batch_size = 256
per_device_train_batch_size = 1
per_device_eval_batch_size = 2
replication_factor = 4
gradient_accumulation = int(global_batch_size / per_device_train_batch_size / replication_factor)

In [23]:
training_args = IPUTrainingArguments(output_dir="./outputs",
                                     do_train=True,
                                     do_eval=True,
                                     per_device_train_batch_size=per_device_train_batch_size,
                                     per_device_eval_batch_size=per_device_eval_batch_size,
                                     gradient_accumulation_steps=gradient_accumulation,
                                     learning_rate=2e-4,
                                     num_train_epochs=2,
                                     logging_steps=25,
                                     dataloader_num_workers=32,
                                     dataloader_drop_last=True,
                                     resume_from_checkpoint=True,
                                     pod_type="pod16",
                                     save_steps=300,
                                     report_to="none",
                                    )

## 4. Training Loop

In [24]:
from squad_preprocessing import PadCollate

Now we create the `IPUTrainer` from `optimum-graphcore` to train our model on the IPU:

In [25]:
trainer = IPUTrainer(model=model,
                     ipu_config=ipu_config,
                     args=training_args, 
                     train_dataset=train_dataset,
                     eval_dataset=validation_features,
                    )

Overriding IPU config: gradient_accumulation_steps=64
-------------------- Device Allocation --------------------
Embedding --> IPU 0
Encoder 0  --> IPU 0
Encoder 1  --> IPU 0
Encoder 2  --> IPU 0
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Encoder 6  --> IPU 1
Encoder 7  --> IPU 1
Encoder 8  --> IPU 1
Encoder 9  --> IPU 1
Encoder 10 --> IPU 2
Encoder 11 --> IPU 2
Encoder 12 --> IPU 2
Encoder 13 --> IPU 2
Encoder 14 --> IPU 2
Encoder 15 --> IPU 2
Encoder 16 --> IPU 2
Encoder 17 --> IPU 3
Encoder 18 --> IPU 3
Encoder 19 --> IPU 3
Encoder 20 --> IPU 3
Encoder 21 --> IPU 3
Encoder 22 --> IPU 3
Encoder 23 --> IPU 3
QA Outputs --> IPU 3
-----------------------------------------------------------


In [26]:
trainer.train(resume_from_checkpoint=False)

Compiling Model...
Graph compilation: 100%|██████████| 100/100 [08:03<00:00]
Compiled/Loaded model in 630.04055990302 secs
***** Running training *****
  Num examples = 88524
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Device Iterations = 1
  Replication Factor = 4
  Gradient Accumulation steps = 64
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Total optimization steps = 690


  0%|          | 0/690 [00:00<?, ?it/s]

{'loss': 2.9382, 'learning_rate': 0.0001927536231884058, 'epoch': 0.07}
{'loss': 1.4743, 'learning_rate': 0.0001855072463768116, 'epoch': 0.14}
{'loss': 1.2007, 'learning_rate': 0.0001782608695652174, 'epoch': 0.22}
{'loss': 1.1122, 'learning_rate': 0.0001710144927536232, 'epoch': 0.29}
{'loss': 0.9775, 'learning_rate': 0.000163768115942029, 'epoch': 0.36}
{'loss': 0.9398, 'learning_rate': 0.0001565217391304348, 'epoch': 0.43}
{'loss': 1.0378, 'learning_rate': 0.00014927536231884058, 'epoch': 0.51}
{'loss': 1.1629, 'learning_rate': 0.00014202898550724638, 'epoch': 0.58}
{'loss': 0.9239, 'learning_rate': 0.0001347826086956522, 'epoch': 0.65}
{'loss': 1.0644, 'learning_rate': 0.00012753623188405797, 'epoch': 0.72}
{'loss': 0.9864, 'learning_rate': 0.00012028985507246378, 'epoch': 0.8}


Saving model checkpoint to ./outputs/checkpoint-300


{'loss': 0.897, 'learning_rate': 0.00011304347826086956, 'epoch': 0.87}


...

-------------------- Device Allocation --------------------
Embedding --> IPU 0
Encoder 0  --> IPU 0
Encoder 1  --> IPU 0
Encoder 2  --> IPU 0
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Encoder 6  --> IPU 1
Encoder 7  --> IPU 1
Encoder 8  --> IPU 1
Encoder 9  --> IPU 1
Encoder 10 --> IPU 2
Encoder 11 --> IPU 2
Encoder 12 --> IPU 2
Encoder 13 --> IPU 2
Encoder 14 --> IPU 2
Encoder 15 --> IPU 2
Encoder 16 --> IPU 2
Encoder 17 --> IPU 3
Encoder 18 --> IPU 3
Encoder 19 --> IPU 3
Encoder 20 --> IPU 3
Encoder 21 --> IPU 3
Encoder 22 --> IPU 3
Encoder 23 --> IPU 3
QA Outputs --> IPU 3
-----------------------------------------------------------
Configuration saved in ./outputs/checkpoint-600/ipu_config.json


{'loss': 0.6963, 'learning_rate': 1.8840579710144928e-05, 'epoch': 1.81}
{'loss': 0.4872, 'learning_rate': 1.1594202898550725e-05, 'epoch': 1.88}
{'loss': 0.9261, 'learning_rate': 4.347826086956522e-06, 'epoch': 1.96}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 536.9026, 'train_samples_per_second': 328.998, 'train_steps_per_second': 1.285, 'train_loss': 0.9179919698963994, 'epoch': 2.0}


TrainOutput(global_step=690, training_loss=0.9179919698963994, metrics={'train_runtime': 536.9026, 'train_samples_per_second': 328.998, 'train_steps_per_second': 1.285, 'train_loss': 0.9179919698963994, 'epoch': 2.0})

After training, we save the model weights to disk.

In [27]:
trainer.save_model()

Saving model checkpoint to ./outputs
-------------------- Device Allocation --------------------
Embedding --> IPU 0
Encoder 0  --> IPU 0
Encoder 1  --> IPU 0
Encoder 2  --> IPU 0
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 1
Encoder 6  --> IPU 1
Encoder 7  --> IPU 1
Encoder 8  --> IPU 1
Encoder 9  --> IPU 1
Encoder 10 --> IPU 2
Encoder 11 --> IPU 2
Encoder 12 --> IPU 2
Encoder 13 --> IPU 2
Encoder 14 --> IPU 2
Encoder 15 --> IPU 2
Encoder 16 --> IPU 2
Encoder 17 --> IPU 3
Encoder 18 --> IPU 3
Encoder 19 --> IPU 3
Encoder 20 --> IPU 3
Encoder 21 --> IPU 3
Encoder 22 --> IPU 3
Encoder 23 --> IPU 3
QA Outputs --> IPU 3
-----------------------------------------------------------
Configuration saved in ./outputs/ipu_config.json


## 5. Validation

We will now take the model we just trained on the training data and run validation on the SQuAD validation dataset. The model will run on a 4-IPU pipeline that we will replicate 4 times.

We loop over all the validation data examples and get the `raw_predictions` for the start and end positions of where the answer to the question lies in the text passage for each one.

In [28]:
eval_output = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `PoptorchPipelinedBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
Compiling Model...
Graph compilation: 100%|██████████| 100/100 [01:49<00:00]
Compiled/Loaded model in 175.55460721894633 secs
***** Running Prediction *****
  Num examples = 10784
  Batch size = 32


  0%|          | 0/337 [00:00<?, ?it/s]

In [29]:
from datasets import load_metric
from squad_preprocessing import postprocess_qa_predictions

In [30]:
raw_predictions = []
raw_predictions.append(eval_output.predictions[0].astype(float))
raw_predictions.append(eval_output.predictions[1].astype(float))

In [31]:
raw_predictions[0].shape

(10784, 384)

In [32]:
validation_features

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 10784
})

We now post-process the raw predictions to the question answering task to get the best prediction that's valid for each one.

In [33]:
final_predictions = postprocess_qa_predictions(datasets["validation"],
                                               validation_features,
                                               raw_predictions)

Post-processing 10570 example predictions split into 10784 features.


  0%|          | 0/10570 [00:00<?, ?it/s]

In [34]:
metric = load_metric("squad")
formatted_predictions = [{"id": k, "prediction_text": v}
                         for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]}
              for ex in datasets["validation"]]
metrics = metric.compute(predictions=formatted_predictions, references=references)
print(metrics)

{'exact_match': 84.46546830652791, 'f1': 90.89570672930324}


We obtain here a good validation score for SQuADv1.

| BERT-Large                             | Exact Match | F1 Score |
|----------------------------------------|:-----------:|:--------:|
| Reference (Devling et al. 2018)        | 84.1        | 90.9     |
| IPU-POD16 with IPU pre-trained weights | 84.5        | 91.0     |

## 6. Inference

We can now use our fine-tuned model to answer questions. Let's start by defining a task:

In [35]:
# Define task
question = "What speed-up can one expect from using sequence packing for training BERT on IPU?"
answer_text = "We find that at sequence length 512 padding tokens represent in excess of 50% of the Wikipedia" \
              "dataset used for pretraining BERT (Bidirectional Encoder Representations from Transformers)." \
             "Therefore by removing all padding we achieve a 2x speed-up in terms of sequences/sec." \
             "To exploit this characteristic of the dataset," \
             "we develop and contrast two deterministic packing algorithms."

Let's get the model inputs ready and create our model. We'll import the weights from the pre-trained, fine-tuned BERT model from the previous sections:

In [36]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_encoding = tokenizer.encode_plus((question, answer_text))

# Extract inputs, add batch dimension
input_tensor = torch.tensor(input_encoding["input_ids"]).unsqueeze(0)
attention_tensor= torch.tensor(input_encoding["attention_mask"]).unsqueeze(0)
token_types=torch.tensor(input_encoding["token_type_ids"]).unsqueeze(0)
    
# Get model and load the fine-tuned weights
model = transformers.BertForQuestionAnswering.from_pretrained("./outputs")

Optionally, instead of using the fine-tuned weights we saved in the previous section, you can download fine-tuned weights from the [Graphcore organisation on the HuggingFace Model Hub](https://huggingface.co/Graphcore). 

In [32]:
# model = transformers.BertForQuestionAnswering.from_pretrained("Graphcore/bert-large-uncased-squad11")

We can now solve the task and print the answer to the question:

In [37]:
# Solve task
outputs = model(input_tensor, attention_tensor, token_types)

# Extract answer
answer_start, answer_stop = outputs.start_logits.argmax(), outputs.end_logits.argmax()
answer_ids = input_tensor.squeeze()[answer_start:answer_stop + 1]
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids, skip_special_tokens=True)
answer = tokenizer.convert_tokens_to_string(answer_tokens)

# Print results
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: What speed-up can one expect from using sequence packing for training BERT on IPU?
Answer: 2x


## 7. Uploading to HuggingFace Model Hub

We can share our model to HuggingFace Model Hub and leverage HuggingFace inference API for downstream tasks.

In [2]:
# Make sure you have git-lfs and huggingface-hub
# !apt-get update && apt-get upgrade -y && apt-get install -y git git-lfs 
# !pip install -y huggingface-hub 

You would need to login to your HuggingFace account and get your token. Running `notebook_login()` will launch an interactive cell and link to your HuggingFace token page.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Upload the checkpoint to HuggingFace Model Hub.
model.push_to_hub("<hf-username>/Optimum-Graphcore-Demo")
tokenizer.push_to_hub("<hf-username>/Optimum-Graphcore-Demo")